In [1]:
import os

import pandas as pd
import spatialdata as sd

/lustre/groups/ml01/workspace/lea.zimmermann/software/miniconda3/envs/sddb2/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


→ connected lamindb: scverse/spatialdata-db


In [2]:
uids_to_add = ["101cw", "103uy", "105tt", "103te", "101in"]

In [3]:
SDDB_KEY = "sample"

DATA_PATH = "/lustre/groups/ml01/projects/2024_spatialdata_db/data"
CSV_FILE_PATH = "./metadata_10x.csv"

df = pd.read_csv(CSV_FILE_PATH, sep="\t")

df = df[df["uid"].isin(uids_to_add)]

In [4]:
for index, row in df.iterrows():
    uid = row["uid"]
    meta = row.drop("uid")

    folder_path = None
    for folder in os.listdir(DATA_PATH):
        if folder.startswith(str(uid)):
            folder_path = os.path.join(DATA_PATH, folder)
            break

    if folder_path is None:
        print(f"Folder for UID {uid} not found.")
        continue

    zarr_file = None
    for file in os.listdir(folder_path):
        if file.endswith(".zarr"):
            zarr_file = os.path.join(folder_path, file)
            break

    # if zarr_file is None and 'metadata' not in zarr_file and 'attrs' not in zarr_file:
    #     print(f"Zarr file not found in folder {folder_path}.")
    #     continue

    sdata = sd.read_zarr(zarr_file)

    meta_df = pd.DataFrame([meta])
    index = meta_df.index[0]
    metadata = {k: v[index] for k, v in meta_df.to_dict().items()}
    # sddb_version = spatialdata_db.__version__
    # metadata['spatialdata_db_attrs'] = {
    #     'version': sddb_version
    # }
    sdata.attrs[SDDB_KEY] = metadata

    metadata_zarr_file = os.path.splitext(zarr_file)[0] + "_attrs.zarr"

    sdata.write(metadata_zarr_file, overwrite=True)

    print(f"Processed UID {uid} and updated metadata.")

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside                                                                                         
         /lustre/groups/ml01/projects/2024_spatialdata_db/data/103uy__10X__Visium__Mouse__brain__20230608__v2.1.0/1
         03uy__10X__Visium__Mouse__brain__20230608__v2.1.0_attrs.zarr). Please see the documentation of            
         `is_self_contained()` to understand the implications of working with SpatialData objects that are not     
         self-contained.                                                                                           
INFO     The Zarr backing store has been changed from                                                              
         /lustre/groups/ml01/projects/2024_spatialdata_db/data/103uy__10X__Visium__Mouse__brain__20230608__v2.1.0/1
         03uy__10X__Visium__Mouse__brain__20230608__v2.1.0.zarr the new 